In [2]:
!pip install keras

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 377 kB 835 kB/s eta 0:00:01     |██████████████████████████      | 307 kB 835 kB/s eta 0:00:01


In [1]:
from keras.models import Model
from keras.optimizers import SGD,Adam,RMSprop
# from keras.layers import Dense, Input, LSTM, Embedding,Dropout,Bidirectional,Flatten
from keras.layers import *
import os

# from __future__ import print_function
from keras import backend as K
from keras.engine.topology import Layer
import h5py

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pandas as pd
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
# Position_Embedding
#! -*- coding: utf-8 -*-
#%%

class Position_Embedding(Layer):
 
    def __init__(self, size=None, mode='sum', **kwargs):
        self.size = size #必须为偶数
        self.mode = mode
        super(Position_Embedding, self).__init__(**kwargs)
 
    def call(self, x):
        if (self.size == None) or (self.mode == 'sum'):
            self.size = int(x.shape[-1])
        batch_size,seq_len = K.shape(x)[0],K.shape(x)[1]
        position_j = 1. / K.pow(10000., \
                                 2 * K.arange(self.size / 2, dtype='float32' \
                               ) / self.size)
        position_j = K.expand_dims(position_j, 0)
        position_i = K.cumsum(K.ones_like(x[:,:,0]), 1)-1 #K.arange不支持变长，只好用这种方法生成
        position_i = K.expand_dims(position_i, 2)
        position_ij = K.dot(position_i, position_j)
        position_ij = K.concatenate([K.cos(position_ij), K.sin(position_ij)], 2)
        if self.mode == 'sum':
            return position_ij + x
        elif self.mode == 'concat':
            return K.concatenate([position_ij, x], 2)
 
    def compute_output_shape(self, input_shape):
        if self.mode == 'sum':
            return input_shape
        elif self.mode == 'concat':
            return (input_shape[0], input_shape[1], input_shape[2]+self.size)

In [3]:
# attention

class Attention(Layer):
 
    def __init__(self, nb_head=8, size_per_head=4, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        super(Attention, self).__init__(**kwargs)
 
    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ',
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK',
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV',
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(Attention, self).build(input_shape)
 
    def Mask(self, inputs, seq_len, mode='mul'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
            mask = 1 - K.cumsum(mask, 1)
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            if mode == 'add':
                return inputs - (1 - mask) * 1e12
 
    def call(self, x):
        #如果只传入Q_seq,K_seq,V_seq，那么就不做Mask
        #如果同时传入Q_seq,K_seq,V_seq,Q_len,V_len，那么对多余部分做Mask
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            Q_len,V_len = None,None
        elif len(x) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = x
        #对Q、K、V做线性变换
        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))
        #计算内积，然后mask，然后softmax
        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))
        A = self.Mask(A, V_len, 'add')
        A = K.permute_dimensions(A, (0,3,2,1))
        A = K.softmax(A)
        #输出并mask
        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, Q_len, 'mul')
        return O_seq
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)

In [4]:
def data_load():
    print('loading data ... \n')
    with h5py.File('../../../get_w2v_feat/w2v1_pre_ad/word_test_ad_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))
    print('get data ... \n')
    return data

def load_data2():
    with h5py.File('../../../get_w2v_feat/w2v1_pre_advertiser/word_test_advertiser_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))
    return data
def load_data3():
    with h5py.File('../../../get_w2v_feat/w2v1_pre_creative_id/word_test_creative_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))
    return data

def load_data4():
    with h5py.File('../../../get_w2v_feat/w2v1_pre_industry/word_test_industry_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))
    return data
def load_data5():
    with h5py.File('../../../get_w2v_feat/w2v1_pre_product_id/word_test_product_id_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))
    return data

In [5]:
# 加载模型
from keras.utils import CustomObjectScope
with CustomObjectScope({'Attention': Attention}):
    lstm_model_path_age1 = '../../../w2v_model5_lstm/attention_comb/cross_multi_attention_aver_age_adm_w200_1.310-0.463-001-1.377-0.438.hdf5'
    model_age1 = keras.models.load_model(lstm_model_path_age1)
    lstm_model_path_age2 = '../../../w2v_model5_lstm/attention_comb/cross_multi_attention_aver_age_adm_w200_1.316-0.463-001-1.378-0.437.hdf5'
    model_age2 = keras.models.load_model(lstm_model_path_age2)
    lstm_model_path_age3 = '../../../w2v_model5_lstm/attention_comb/cross_multi_attention_aver_age_adm_w200_1.324-0.458-001-1.377-0.438.hdf5'
    model_age3 = keras.models.load_model(lstm_model_path_age3)
    lstm_model_path_age4 = '../../../w2v_model5_lstm/attention_comb/cross_multi_attention_aver_age_adm_w200_1.326-0.456-001-1.379-0.437.hdf5'
    model_age4 = keras.models.load_model(lstm_model_path_age4)
    lstm_model_path_age5 = '../../../w2v_model5_lstm/attention_comb/cross_multi_attention_aver_age_adm_w200_1.331-0.456-001-1.378-0.438.hdf5'
    model_age5 = keras.models.load_model(lstm_model_path_age5)

#     lstm_model_path_gender1 = '../../../w2v_model5_lstm/attention_comb/cross_multi_attention_aver_gender_adm_w200_0.169-0.941-001-0.181-0.936.hdf5'
#     model_gender1 = keras.models.load_model(lstm_model_path_gender1)
#     lstm_model_path_gender2 = '../../../w2v_model5_lstm/attention_comb/cross_multi_attention_aver_gender_adm_w200_0.171-0.939-001-0.180-0.936.hdf5'
#     model_gender2 = keras.models.load_model(lstm_model_path_gender2)
#     lstm_model_path_gender3 = '../../../w2v_model5_lstm/attention_comb/cross_multi_attention_aver_gender_adm_w200_0.171-0.941-001-0.181-0.935.hdf5'
#     model_gender3 = keras.models.load_model(lstm_model_path_gender3)
#     lstm_model_path_gender4 = '../../../w2v_model5_lstm/attention_comb/cross_multi_attention_aver_gender_adm_w200_0.176-0.938-001-0.180-0.936.hdf5'
#     model_gender4 = keras.models.load_model(lstm_model_path_gender4)
#     lstm_model_path_gender5 = '../../../w2v_model5_lstm/attention_comb/cross_multi_attention_aver_gender_adm_w200_0.174-0.939-001-0.181-0.935.hdf5'
#     model_gender5 = keras.models.load_model(lstm_model_path_gender5)
    







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:

print('load data...')
data_ad = data_load()
data_advertiser = load_data2()
data_creative = load_data3()
data_industry = load_data4()
data_product_id = load_data5()


data_age = [data_ad,data_advertiser,data_creative]
data_gender = [data_ad,data_advertiser,data_creative,data_industry,data_product_id]

# print('pre gender ...')

# probs_gender1 = model_gender1.predict(data_gender, verbose=0)
# probs_gender2 = model_gender2.predict(data_gender, verbose=0)
# probs_gender3 = model_gender3.predict(data_gender, verbose=0)
# probs_gender4 = model_gender4.predict(data_gender, verbose=0)
# probs_gender5 = model_gender5.predict(data_gender, verbose=0)

# pre_gender = (probs_gender1 + probs_gender2+probs_gender3+probs_gender4+probs_gender5)/5

# pre_gender = np.array(probs_gender>0.5).astype('int')
# pre_gender = pre_gender + 1


print('pre age ...')

probs_age1 = model_age1.predict(data_age, verbose=0)
probs_age2 = model_age2.predict(data_age, verbose=0)
probs_age3 = model_age3.predict(data_age, verbose=0)
probs_age4 = model_age4.predict(data_age, verbose=0)
probs_age5 = model_age5.predict(data_age, verbose=0)

pre_age = (probs_age1+probs_age2+probs_age3+probs_age4+probs_age5)/5


load data...
loading data ... 

get data ... 

pre age ...


NameError: name 'probs_age' is not defined

In [26]:
pre_age = (probs_age1+probs_age2+probs_age3+probs_age4+probs_age5)/5

In [28]:
np.shape(pre_age)

(1000000, 10)

In [31]:
res = pd.DataFrame(pre_age)
res.to_csv('submission_age_pro.csv',index=False)

In [7]:
pre_age = np.reshape(np.argmax(pre_age, axis=1),[-1])
pre_age = pre_age + 1

In [8]:
test_user_id = np.array(range(3000001,4000001))
res = pd.DataFrame(test_user_id)
res.columns = [['user_id']]

res['predicted_age']=pre_age
# res['predicted_gender']=pre_gender

In [9]:
res.to_csv('submission2.csv',index=False)

In [12]:
gender = pd.read_csv('submission.csv')

In [16]:
gender

,user_id,predicted_gender
0,3000001,0.005375
1,3000002,0.997314
2,3000003,0.790318
3,3000004,0.194321
4,3000005,0.007194
...,...,...
999995,3999996,0.011047
999996,3999997,0.045251
999997,3999998,0.010197
999998,3999999,0.008800


In [17]:
pre_gender = gender['predicted_gender'].values

In [18]:
pre_gender

array([0.00537505, 0.9973135 , 0.79031765, ..., 0.01019735, 0.00879961,
       0.00307634])

In [19]:
pre_gender = np.array(pre_gender>0.5).astype('int')
pre_gender = pre_gender + 1

In [20]:
pre_gender

array([1, 2, 2, ..., 1, 1, 1])

In [22]:
res['predicted_gender']=pre_gender

In [24]:
res.to_csv('submission.csv',index=False)